# **Task 3: Linear Regression via Least Squares**

We built a **linear regression model** to predict house prices based on three features:  
**square footage**, **number of bedrooms**, and **age of the house**.

---

### **Summary of What We Did**

- We set up the linear model  
  $$
  X\beta = y,
  $$
  where $X$ contains the house features and $y$ represents the prices (in \$1000s).  
- We solved for the regression coefficients $\beta$ using both:
  - the **least squares method** (`scipy.linalg.lstsq`), and  
  - the **direct normal equation solution** (`scipy.linalg.solve`).
- We used the model to predict the price of a new house with **2400 sq ft, 3 bedrooms, and 20 years old**.  
- Finally, we compared both solution methods and evaluated the model fit.

---

### **Results**

- **Least Squares Coefficients:**  
  $$
  \hat{\beta} =
  \begin{bmatrix}
  3.2875 \\ -1628.75 \\ -77.75
  \end{bmatrix}
  $$
  where:
  - $ \beta_1 $: effect of square footage  
  - $ \beta_2 $: effect of number of bedrooms  
  - $ \beta_3 $: effect of house age  

- **Model Performance:**  
  - Residual Sum of Squares (RSS): **25.00**  
  - Root Mean Squared Error (RMSE): **2.50**

- **Predicted Price for [2400 sq ft, 3 beds, 20 yrs]:**  
  $$
  \hat{y}_{\text{new}} = 1448.75 \text{ (in \$1000)}
  $$

- **Comparison:**  
  Both `lstsq` and the direct normal equation solution produced nearly identical results  
  $L₂ \;\text{gap} ≈ 5.6 \times 10^{-9}$, confirming numerical consistency.  

---

### **Interpretation**

- The **positive coefficient for square footage (3.29)** means that, on average, increasing the house size by one square foot increases the predicted price by about **\$3,290** (since values are in \$1000s).  
- The **negative coefficients** for **bedrooms (-1628.75)** and **age (-77.75)** are not meaningful in this small dataset because the features are **strongly correlated**.  
  In practice, adding bedrooms usually *increases* price, but here the model compensates for overlapping effects between features (collinearity).  
- The small **RSS (25)** and **RMSE (2.5)** simply indicate that the regression fits these four points very closely, not that it generalizes well.  
- Overall, both the **least-squares** and **direct-solve** methods gave the same solution difference less than $  10^{-8}$, confirming the correctness of the computation, but the **signs of coefficients** should not be overinterpreted due to multicollinearity and the very limited dataset.


In [2]:
import numpy as np  # I import numpy for vector and matrix computations
from scipy.linalg import lstsq, solve  # I import least-squares and direct solver utilities from SciPy

X = np.array([  # I define the feature matrix with columns: [square_footage, bedrooms, age]
    [2100, 3, 20],  # I enter row 1 from the assignment table
    [2500, 4, 15],  # I enter row 2 from the assignment table
    [1800, 2, 30],  # I enter row 3 from the assignment table
    [2200, 3, 25],  # I enter row 4 from the assignment table
], dtype=float)  # I force floating point dtype to avoid integer division issues

y = np.array([460, 540, 330, 400], dtype=float)  # I define the target price vector (in $1000s) as floats

beta_ls, residuals, rank, s = lstsq(X, y)  # I solve X beta ≈ y via least squares and collect diagnostics
y_fit = X @ beta_ls  # I compute the fitted responses using the least-squares coefficients
rss = np.sum((y - y_fit) ** 2)  # I compute the residual sum of squares for model fit assessment
rmse = np.sqrt(rss / X.shape[0])  # I compute RMSE using the sample size as a simple summary error measure

x_new = np.array([2400, 3, 20], dtype=float)  # I create the new house feature vector for prediction
y_new_pred = float(x_new @ beta_ls)  # I compute the predicted price for the new house (in $1000s)

XtX = X.T @ X  # I form the normal-equation matrix X^T X to compare with a direct solver
Xty = X.T @ y  # I form the right-hand side vector X^T y for the normal equations
beta_direct = solve(XtX, Xty, assume_a='pos')  # I solve the normal equations directly (works if XtX is invertible)
coef_gap = np.linalg.norm(beta_ls - beta_direct, ord=2)  # I measure the L2 difference between the two coefficient estimates

print("=== Least Squares Solution (scipy.linalg.lstsq) ===")  # I label the section for clarity
print("beta (coefficients) =", beta_ls)  # I print the estimated coefficients for sqft, bedrooms, and age
print("rank(X) =", rank)  # I print the numerical rank of X reported by lstsq
print("singular values of X =", s)  # I print the singular values from the least-squares routine
print(f"RSS = {rss:.4f}")  # I print the residual sum of squares to summarize lack of fit
print(f"RMSE = {rmse:.4f}")  # I print RMSE as a scale-aware error metric
print()  # I add a blank line for readability

print("=== Prediction for New House [2400 sqft, 3 beds, 20 yrs] ===")  # I label the prediction section
print(f"Predicted price (in $1000s) = {y_new_pred:.3f}")  # I report the predicted price for the new house
print()  # I add a blank line for readability

print("=== Comparison with Direct Solve of Normal Equations ===")  # I label the comparison section
print("beta (normal equations) =", beta_direct)  # I print coefficients from the direct solver
print(f"L2 gap between lstsq and normal-equation solution = {coef_gap:.3e}")  # I report numerical difference between methods
print("Note: lstsq is generally preferred for numerical stability, especially when X^T X is ill-conditioned.")  # I explain why least squares is the safer default


=== Least Squares Solution (scipy.linalg.lstsq) ===
beta (coefficients) = [    3.2875 -1628.75     -77.75  ]
rank(X) = 3
singular values of X = [4.32919549e+03 1.59510474e+01 2.89623364e-02]
RSS = 25.0000
RMSE = 2.5000

=== Prediction for New House [2400 sqft, 3 beds, 20 yrs] ===
Predicted price (in $1000s) = 1448.750

=== Comparison with Direct Solve of Normal Equations ===
beta (normal equations) = [    3.2875     -1628.75000001   -77.75      ]
L2 gap between lstsq and normal-equation solution = 5.608e-09
Note: lstsq is generally preferred for numerical stability, especially when X^T X is ill-conditioned.
